In [ ]:
!pip install pybatfish==2019.11.1.276

In [4]:
from pybatfish.client.session import Session  # noqa: F401

In [5]:
bf = Session(host="localhost")

BatfishException: Coordinator returned failure: Failed to visit templates dir: questions